# Traffic Sign Recognition Model
## Environment Initialisation

In [1]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")
!ls
import os
os.chdir("My Drive/[ Masters ] - Deep Learning Proj/Model")
!pip install import_ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [3]:
# Libraries
import os
import sys
import tensorflow as tf
import numpy as np
import math
IntialiseEnv()
import import_ipynb
import pickle
from sklearn.model_selection import train_test_split

# from Data_Preparation.Data_Preparation_Croatian import LoadCroatianTrainDataSet
# from Data_Preparation.Data_Preparation_Croatian import LoadCroatianTestDataSet

## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [4]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0
/device:GPU:0
Num GPUs Available:  1


In [5]:
import cv2
from skimage.util import random_noise
from skimage import exposure
def preprocess_(img):
  img=img.reshape(60,60)
  img=cv2.blur(img,(2,2))
  img=random_noise(img)
  v_min, v_max = np.percentile(img, (0.2, 99.8))
  img = exposure.rescale_intensity(img, in_range=(v_min, v_max))
  # img=exposure.adjust_gamma(img, gamma=0.4, gain=0.9)
  
  img_final=img.reshape(60,60,1)
  return img_final

In [6]:
def ReadTrainDataSet():
    print("Reading Train Pre-processed DataSet")
    processedTrainDataSet = '../DataSet/Processed_DataSet/CroatianTrainDataSet.pkl'
    isProcessedTrainDataSetExits= os.path.exists(processedTrainDataSet)
    train_image_array, train_image_labels = None, None
    train_image_array_final=[]
    train_image_labels_final=[]
    
    if isProcessedTrainDataSetExits:
        print("Loading Processed Train DataSet from Processed_DataSet/CroatianTrainDataSet.pkl")
        file = open(processedTrainDataSet, 'rb')
        train_image_array, train_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Train DataSet.")
        
        # for i in range(0,len(train_image_array)):
        #   pre_process_img=preprocess_(train_image_array[i])
        #   train_image_array_final.append(pre_process_img)
        #   train_image_array_final.append(train_image_array[i])
        #   # print(train_image_array[i].shape,pre_process_img.shape)
        #   train_image_labels_final.append(train_image_labels[i])
        #   train_image_labels_final.append(train_image_labels[i])
        # train_image_labels_final=np.array(train_image_labels_final)
        # train_image_array_final=np.array(train_image_array_final)
        


        
    else:
        print("Processed_DataSet/CroatianTrainDataSet.pkl file does not exist")
        print("Loading Train DataSet ... This may take a while.")
        train_image_array, train_image_labels =  LoadCroatianTrainDataSet()
        file = open(processedTrainDataSet, 'wb')
        pickle.dump((train_image_array, train_image_labels), file, protocol=4)
        file.close()
        print("Saving pre-processed train DataSets in Processed_DataSet/CroatianTrainDataSet.pkl")
    return train_image_array, train_image_labels

In [7]:
def ReadTestDataSet():
    processedTestDataSet = '../DataSet/Processed_DataSet/CroatianTestDataSet.pkl'
    isProcessedTestDataSetExits= os.path.exists(processedTestDataSet)
    test_image_array, test_image_labels = None, None
    
    if isProcessedTestDataSetExits:
        print("Loading Processed Test DataSet from Processed_DataSet/CroatianTestDataSet.pkl")
        file = open(processedTestDataSet, 'rb')
        test_image_array, test_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Test DataSet.")
    else:
        print("Processed_DataSet/CroatianTestDataSet.pkl file does not exist")
        print("Loading Test DataSet ... This may take a while.")
        test_image_array, test_image_labels = LoadCroatianTestDataSet()
        file = open(processedTestDataSet, 'wb')
        pickle.dump((test_image_array, test_image_labels), file)
        file.close()
        print("Saving pre-processed test DataSets in Processed_DataSet/CroatianTestDataSet.pkl")
    return test_image_array, test_image_labels

In [8]:
train_image_array, train_image_labels= ReadTrainDataSet()

Reading Train Pre-processed DataSet
Loading Processed Train DataSet from Processed_DataSet/CroatianTrainDataSet.pkl
Done Loading Train DataSet.


In [9]:
print(train_image_array.shape)

(4044, 60, 60, 1)


In [10]:
test_image_array, test_image_labels= ReadTestDataSet()

Loading Processed Test DataSet from Processed_DataSet/CroatianTestDataSet.pkl
Done Loading Test DataSet.


In [11]:
print(test_image_array.shape)

(1784, 60, 60, 1)


## Train-Split 
Split Train images dataset into two splits: training and validation respectively 90-10

In [12]:
image_train, image_valid,label_train, label_valid = train_test_split(train_image_array, train_image_labels,stratify=train_image_labels,test_size=0.1,random_state=43)

In [13]:
print(image_train.shape)

(3639, 60, 60, 1)


In [14]:
del train_image_array
del train_image_labels

## Hyper-parameters Tuning

In [15]:
# Hyper-parameters tuning
kernel_2 = (3,3)
pooling = (2,2)
dropout = 0.3
num_classes = 31

## Model Architecture

In [16]:
import tensorflow as tf
import keras

from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,SpatialDropout2D
from keras.layers import Conv2D, MaxPool2D, Add,AveragePooling2D
from keras.models import Sequential
from keras.layers import Conv2D, SeparableConv2D,BatchNormalization,Dropout,MaxPool2D,Flatten,Dense
from keras.layers import concatenate

from keras import initializers
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.regularizers import l2

In [17]:
weight_decay=1E-4
from tensorflow.keras.regularizers import l2

def model_():
    model = None
    tf.initializers.Orthogonal(gain=1.0, seed=None)
    model=Sequential()

    model.add(Conv2D(32,(5,5), input_shape=(60,60,1), strides = 1, padding='valid',activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3), activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    # model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(Conv2D(64,(5,5), strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3), activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    # model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(Conv2D(256,(5,5), strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    # model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(512, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    # model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation = 'softmax'))
    return model

In [18]:
# def model_():
#     model = None
#     tf.initializers.Orthogonal(gain=1.0, seed=None)
#     model=Sequential()

#     model.add(Conv2D(16,kernel_2, input_shape=(60,60,1), strides = 1, padding='valid',activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
#     model.add(BatchNormalization())
#     model.add(Conv2D(32,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
#     model.add(MaxPool2D(pooling))
#     # model.add(Dropout(dropout))
#     model.add(BatchNormalization())

#     model.add(Conv2D(64,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
#     model.add(BatchNormalization())
#     model.add(Conv2D(128,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
#     model.add(MaxPool2D(pooling))
#     # model.add(Dropout(dropout))
#     model.add(BatchNormalization())

#     model.add(Conv2D(256,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
#     model.add(BatchNormalization())
#     model.add(Conv2D(256,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
#     model.add(MaxPool2D(pooling))
#     # model.add(Dropout(dropout))
#     model.add(BatchNormalization())
#     model.add(Flatten())
#     model.add(Dense(512, activation = tf.nn.relu))
#     # model.add(Dropout(0.4))
#     model.add(Dense(num_classes, activation = 'softmax'))
#     return model

In [19]:
from tensorflow import keras
model = model_()

## Model Optimizer 

In [20]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2,  verbose=1, min_delta=1e-4, min_lr=1e-20)

In [21]:


model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            # preprocessing_function=get_random_eraser(v_l=0, v_h=255),
                           
                            rotation_range=10.,)

In [23]:
x=aug.flow(image_train, np.array(label_train), batch_size=32)

## Model Training

In [24]:
history = model.fit(x, epochs=30, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/30
114/114 [==============================] - 4s 35ms/step - loss: 97.2760 - sparse_categorical_accuracy: 0.4955 - val_loss: 79.2372 - val_sparse_categorical_accuracy: 0.0321
Epoch 2/30
114/114 [==============================] - 3s 29ms/step - loss: 61.7337 - sparse_categorical_accuracy: 0.7356 - val_loss: 51.9513 - val_sparse_categorical_accuracy: 0.0321
Epoch 3/30
114/114 [==============================] - 3s 29ms/step - loss: 39.3600 - sparse_categorical_accuracy: 0.8439 - val_loss: 34.0360 - val_sparse_categorical_accuracy: 0.1333
Epoch 4/30
114/114 [==============================] - 3s 29ms/step - loss: 25.2516 - sparse_categorical_accuracy: 0.8777 - val_loss: 22.5963 - val_sparse_categorical_accuracy: 0.1877
Epoch 5/30
114/114 [==============================] - 3s 29ms/step - loss: 16.3057 - sparse_categorical_accuracy: 0.8978 - val_loss: 14.6441 - val_sparse_categorical_accuracy: 0.4790
Epoch 6/30
114/114 [==============================] - 3s 29ms/step - loss: 10.5989 - 

In [31]:
history = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
114/114 [==============================] - 4s 31ms/step - loss: 0.1865 - sparse_categorical_accuracy: 0.9915 - val_loss: 0.2230 - val_sparse_categorical_accuracy: 0.9852
Epoch 2/5
114/114 [==============================] - 3s 30ms/step - loss: 0.2055 - sparse_categorical_accuracy: 0.9860 - val_loss: 0.2018 - val_sparse_categorical_accuracy: 0.9778
Epoch 3/5
114/114 [==============================] - ETA: 0s - loss: 0.2189 - sparse_categorical_accuracy: 0.9832
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
114/114 [==============================] - 3s 29ms/step - loss: 0.2189 - sparse_categorical_accuracy: 0.9832 - val_loss: 0.2152 - val_sparse_categorical_accuracy: 0.9852
Epoch 4/5
114/114 [==============================] - 3s 29ms/step - loss: 0.1842 - sparse_categorical_accuracy: 0.9934 - val_loss: 0.1707 - val_sparse_categorical_accuracy: 0.9951
Epoch 5/5
114/114 [==============================] - 3s 29ms/step - loss: 0.1726 - sparse_catego

In [ ]:
history = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

In [33]:
history = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1678 - sparse_categorical_accuracy: 0.9967 - val_loss: 0.1596 - val_sparse_categorical_accuracy: 0.9975
Epoch 2/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1604 - sparse_categorical_accuracy: 0.9984 - val_loss: 0.1595 - val_sparse_categorical_accuracy: 0.9975
Epoch 3/5
114/114 [==============================] - 3s 29ms/step - loss: 0.1558 - sparse_categorical_accuracy: 0.9986 - val_loss: 0.1575 - val_sparse_categorical_accuracy: 0.9951
Epoch 4/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1531 - sparse_categorical_accuracy: 0.9981 - val_loss: 0.1536 - val_sparse_categorical_accuracy: 0.9951
Epoch 5/5
114/114 [==============================] - 3s 29ms/step - loss: 0.1481 - sparse_categorical_accuracy: 0.9989 - val_loss: 0.1516 - val_sparse_categorical_accuracy: 0.9951


In [ ]:
history = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

In [40]:
history = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1331 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.1379 - val_sparse_categorical_accuracy: 0.9975
Epoch 2/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1299 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1360 - val_sparse_categorical_accuracy: 0.9975
Epoch 3/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1287 - sparse_categorical_accuracy: 0.9986 - val_loss: 0.1441 - val_sparse_categorical_accuracy: 0.9901
Epoch 4/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1261 - sparse_categorical_accuracy: 0.9992 - val_loss: 0.1335 - val_sparse_categorical_accuracy: 0.9975
Epoch 5/5
114/114 [==============================] - 3s 30ms/step - loss: 0.1233 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.1301 - val_sparse_categorical_accuracy: 0.9975


In [ ]:
history = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
114/114 [==============================] - 3s 30ms/step - loss: 0.0714 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0822 - val_sparse_categorical_accuracy: 0.9951
Epoch 2/5
114/114 [==============================] - 3s 29ms/step - loss: 0.0710 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0824 - val_sparse_categorical_accuracy: 0.9926
Epoch 3/5
 23/114 [=====>........................] - ETA: 2s - loss: 0.0699 - sparse_categorical_accuracy: 1.0000

## Model Evaluation

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

In [41]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

7/7 [==============================] - 0s 32ms/step - loss: 0.1437 - sparse_categorical_accuracy: 0.9950


[0.14365625381469727, 0.9949551820755005]

In [34]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

7/7 [==============================] - 0s 32ms/step - loss: 0.1669 - sparse_categorical_accuracy: 0.9955


[0.16686005890369415, 0.9955157041549683]

In [26]:
xxx

NameError: ignored

## Visualization

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Model Summary 
Number of parameters used in the model architecture

In [ ]:
print(model.summary())

## Saving Trained .h5 Model

In [35]:
def saveCNNModel(model_name):
    model.save(model_name)

In [37]:
saveCNNModel('Trained_Models/Croatian_Winning_99_55_.h5')

In [ ]:
model._set_inputs(image_train,np.array(label_train))

## Loading Saved Trained .h5 Model

In [ ]:
del model

In [ ]:
def load_model():
    Model = tf.keras.models.load_model('Trained_Models/Croatian_Winning_99_27.h5')
    return Model

In [ ]:
model= load_model()

In [ ]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

## Computation Avg. Processing Time

In [ ]:
import time
t1=time.time()
model.predict(test_image_array)
t2=time.time()
print("Average Processing time: ", ((t2-t1)/12630)*1000, " ms")